In [42]:
# Import libraries
import spacy
from nltk.corpus import stopwords
from collections import Counter
import nltk
from scispacy.linking import EntityLinker
import scispacy
from tqdm import tqdm
import re
import unicodedata
import os
import logging
import chardet
from collections import OrderedDict

%matplotlib inline
import matplotlib.pyplot as plt

In [43]:
# nltk.download('punkt')
# nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

nlp = spacy.load("en_core_sci_md")
nlp.add_pipe("scispacy_linker", last=True)

print("Setup completed.")

Setup completed.


In [44]:
# reassemble_hyphenated_words
def reassemble_hyphenated_words(text):
    return re.sub(r'(\w+)-\s*\n(\w+)', r'\1\2', text)

def remove_urls(text):
    return re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)

def remove_emails(text):
    email_pattern = r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b'
    return re.sub(email_pattern, '', text, flags=re.IGNORECASE)

def remove_figures_tables(text):
    return re.sub(r'\b(figures?|tables?)\b', '', text)

def remove_numerical_references(text):
    return re.sub(r'\[\d+\]', '', text)

def remove_citations(text):
    patterns = [
        r'\b[A-Z][a-z]+\s+[A-Z][a-z]+\s+[A-Z][a-z]+\s+et\s+al\.',
        r'\b[A-Z][a-z]+\s+[A-Z][a-z]+\s+et\s+al\.',
        r'\b[A-Z][a-z]+\s+et\s+al\.',
        r'\(.*?et al\..*?\d{4}.*?\)',
        r'\[.*?\]',
        r'\(\d{4}[a-z]?(?:,\s*\d{4}[a-z]?)*\)',
        r'^.*?\d{4};.*?:\s*\d+.*?$',
        r'^.*?\d{4};.*?:\s*\d+.*?$',  # Matches journal info like "2024;258: 119– 129"
    ]

    for pattern in patterns:
        text = re.sub(pattern, '', text, flags=re.MULTILINE | re.IGNORECASE)
    return text

def remove_headers(text):
    # Remove lines that are all uppercase and end with a colon
    text = re.sub(r'^[A-Z\s]+:$', '', text, flags=re.MULTILINE)
    # Remove lines that start with bullet points
    text = re.sub(r'^\s*•.*$', '', text, flags=re.MULTILINE)
    return text

def remove_metadata(text):
    # Remove headers, copyright info, DOI, received/accepted dates
    patterns = [
        r'^.*?©Copyright.*$',
        r'^DOI:.*$',
        r'^Received:.*$',
        r'^Accepted:.*$',
        r'^Address for Correspondence:.*$',
        r'^E-mail:.*$',
        r'^ORCID-ID:.*$',
        r'^\s*\d+\s*$',  # Page numbers
        r'^.*?ORCID:.*$',
        r'^Cite this article as:.*$',
        r'\[\s*[^\w\s]*\s*\]'
    ]
    for pattern in patterns:
        text = re.sub(pattern, '', text, flags=re.MULTILINE)
    return text

def remove_institution_names(text):
    # Remove institution names (this is a simplified approach and may need refinement)
    pattern = r'\*+[A-Z][A-Za-z\s,]+(University|Institute|Hospital|Department|Faculty)[^\n]*'
    return re.sub(pattern, '', text, flags=re.MULTILINE)

def handle_special_characters(text):
    """Handle special characters and Unicode normalization."""
    text = unicodedata.normalize('NFKD', text).encode('ASCII', 'ignore').decode('ASCII')
    char_map = {
        '´': "'", '‘': "'", '’': "'", '“': '"', '”': '"', '–': '-', '—': '-', '…': '...'
    }
    for char, replacement in char_map.items():
        text = text.replace(char, replacement)
    return text

def remove_copyright_info(text):
    patterns = [
        r'^©.*$',
        r'Copyright.*$',
        r'This is an open access article.*$',
    ]
    for pattern in patterns:
        text = re.sub(pattern, '', text, flags=re.MULTILINE | re.IGNORECASE)
    return text

def remove_doi_and_journal_info(text):
    patterns = [
        r'DOI:.*$',
        r'^.*?\d{4};\d+:\d+–\d+',  # Matches journal info like "2024;258: 119– 129"
    ]
    for pattern in patterns:
        text = re.sub(pattern, '', text, flags=re.MULTILINE)
    return text

def remove_artifacts(text):
    # Remove license and DOI information
    text = re.sub(r'BY license \(.*?\)\..*?commons\.org/licenses/by/\d\.\d/\s*\)\.', '', text)
    text = re.sub(r'://doi\.org/\d+\.\d+/[^\s]+', '', text )
    text = re.sub(r'://creativecommons\.org/licenses/by/\d\.\d/', '', text)
    text = re.sub(r'://creativecommons\.org/licenses/by-\w+/\d\.\d/', '', text)
    
    
    # Remove unnecessary symbols
    text = re.sub(r'[⁎\]]', '', text)
    
    # Remove empty parentheses and brackets
    text = re.sub(r'\(\s*\)|\[\s*\]', '', text)
    
    # Remove isolated semicolons
    text = re.sub(r'\s*;\s*', ' ', text)
    
    # Remove extra spaces
    text = re.sub(r'\s+', ' ', text)
    
    return text.strip()

def detect_sentence_boundaries(text):
    """
    Detect sentence boundaries using spaCy.
    """
    doc = nlp(text)
    spacy_sentences = [sent.text for sent in doc.sents]
    return ' '.join(spacy_sentences)

In [45]:
def count_words(text):
    """Count the number of words in the given text."""
    # Split on word boundaries
    words = re.findall(r'\b[\w\'-]+\b', text)
    # Filter out purely numeric "words"
    return len([word for word in words if not word.isdigit()])

In [46]:
def extract_abstract_and_main_text(text):
    # Possible sections indicating the beginning of the abstract
    abstract_start_patterns = [
        r'\bAbstract\b',
        r'\bBackground\b',
        r'\bPurpose\b'
    ]
    
    # Possible sections indicating the end of the abstract
    abstract_end_patterns = [
        r'\bKeywords\b',
        r'\bIntroduction\b'
    ]
    
    # Initialize abstract boundaries
    abstract_start = None
    abstract_end = None
    
    # Search for the start of the abstract using standard patterns
    for pattern in abstract_start_patterns:
        match = re.search(pattern, text, re.IGNORECASE)
        if match:
            abstract_start = match.end()
            break
    
    # If no start pattern is found, assume the abstract starts after any metadata or headers
    if not abstract_start:
        lines = text.splitlines()
        for line in lines:
            if re.search(r'\bAuthor[s]?|Acknowledgment[s]?|Declaration[s]?\b', line, re.IGNORECASE):
                continue
            if len(line.strip()) > 30 and not re.search(r'\[\d+\]', line):
                abstract_start = text.find(line)
                break

    # Exit if we cannot determine the start of the abstract
    if abstract_start is None:
        return "Abstract not found.", text
    
    # Search for the end of the abstract
    for pattern in abstract_end_patterns:
        match = re.search(pattern, text[abstract_start:], re.IGNORECASE)
        if match:
            abstract_end = abstract_start + match.start()
            break
    
    # Default to the end of the text if no clear end pattern is found
    if not abstract_end:
        abstract_end = len(text)
    
    # Extract the abstract content
    abstract = text[abstract_start:abstract_end].strip()
    
    # Extract the main text starting from the end of the abstract
    main_text = text[abstract_end:].strip()

    # Optionally, limit the abstract to a certain number of words
    max_words = 250
    words = abstract.split()
    if len(words) > max_words:
        abstract = ' '.join(words[:max_words])
    
    return abstract, main_text


In [47]:
def preprocess_text(text):
    try:
        print(f"Starting preprocessing...")
        initial_word_count = count_words(text)
        print(f"Initial word count: {initial_word_count}")

        # Extract abstract
        abstract, main_text = extract_abstract_and_main_text(text)
        print(f"Abstract extracted. Abstract word count: {count_words(abstract)}")
        print(f"Main text word count: {count_words(main_text)}")

        # Step 1: Initial text cleaning
        main_text = reassemble_hyphenated_words(main_text)
        print(f"After reassemble_hyphenated_words: {count_words(main_text)} words")

        main_text = remove_figures_tables(main_text)
        print(f"After remove_figures_tables: {count_words(main_text)} words")

        main_text = remove_citations(main_text)
        print(f"After remove_citations: {count_words(main_text)} words")

        main_text = remove_urls(main_text)
        print(f"After remove_urls: {count_words(main_text)} words")

        main_text = remove_emails(main_text)
        print(f"After remove_emails: {count_words(main_text)} words")

        main_text = remove_numerical_references(main_text)
        print(f"After remove_numerical_references: {count_words(main_text)} words")

        main_text = remove_headers(main_text)
        print(f"After remove_headers: {count_words(main_text)} words")

        main_text = remove_metadata(main_text)
        print(f"After remove_metadata: {count_words(main_text)} words")

        main_text = remove_institution_names(main_text)
        print(f"After remove_institution_names: {count_words(main_text)} words")
        
        main_text = remove_copyright_info(main_text)
        print(f"After remove_copyright_info: {count_words(main_text)} words")

        main_text = remove_doi_and_journal_info(main_text)
        print(f"After remove_doi_and_journal_info: {count_words(main_text)} words")

        main_text = remove_artifacts(main_text)
        print(f"After remove_artifacts: {count_words(main_text)} words")

        # Remove extra whitespace
        main_text = ' '.join(main_text.split())
        print(f"After removing extra whitespace: {count_words(main_text)} words")
    
        main_text = handle_special_characters(main_text)
        print(f"After handle_special_characters: {count_words(main_text)} words")

        main_text = detect_sentence_boundaries(main_text)
        print(f"After detect_sentence_boundaries: {count_words(main_text)} words")

        final_word_count = count_words(main_text)
        print(f"Final preprocessed word count: {final_word_count}")

        if initial_word_count > 0:
            percentage_retained = (final_word_count / initial_word_count) * 100
            print(f"Percentage of words retained after cleaning: {percentage_retained:.2f}%")
        else:
            print("Original text contains no words; cannot calculate percentage.")

        return abstract, main_text.strip()

    except Exception as e:
        logging.error(f"Error in preprocess_text: {e}")
        return None, ""

In [48]:
import os
import chardet
from tqdm import tqdm
import logging

def process_files(input_folder_path: str, output_folder_path: str, batch_size: int = 10) -> None:
    if not os.path.exists(output_folder_path):
        os.makedirs(output_folder_path)

    files = [f for f in os.listdir(input_folder_path) if f.endswith('.txt')]
    total_files = len(files)

    for i in range(0, total_files, batch_size):
        batch = files[i:i+batch_size]
        print(f"\nProcessing batch {i//batch_size + 1} of {(total_files-1)//batch_size + 1}")

        for filename in tqdm(batch, desc=f"Batch {i//batch_size + 1}"):
            input_file_path = os.path.join(input_folder_path, filename)
            output_file_path = os.path.join(output_folder_path, f"processed_{filename}")

            print(f"\nProcessing file: {filename}")

            try:
                with open(input_file_path, 'rb') as file:
                    raw_data = file.read()
                    result = chardet.detect(raw_data)
                    detected_encoding = result['encoding']
                    confidence = result['confidence']

                with open(input_file_path, 'r', encoding=detected_encoding or 'utf-8') as file:
                    original_text = file.read()

                # Extract abstract and clean main text
                abstract, cleaned_text = preprocess_text(original_text)

                # Prepare output text
                output_text = f"Abstract:\n{abstract}\n\nMain Text:\n{cleaned_text}"

                # Write the cleaned text and abstract to the output file
                with open(output_file_path, 'w', encoding='utf-8') as file:
                    file.write(output_text)

            except Exception as e:
                logging.error(f"Error processing file {filename}: {str(e)}")

            print("=" * 100)

        print(f"Completed processing batch {i//batch_size + 1}")

    print("All batches processed.")


In [49]:
def identify_frequent_terms(text, n=10, min_length=3):

    # Tokenize the text into words
    words = re.findall(r'\b[a-zA-Z]{' + str(min_length) + r',}\b', text.lower())
    
    # Remove stop words
    words = [word for word in words if word not in stop_words]
    
    # Count word frequencies
    word_freq = Counter(words)
    
    # Return the n most common words with their counts
    return word_freq.most_common(n)

In [50]:
if __name__ == "__main__":
    # input_folder = "/mnt/data/skanda/MSc_IRD_LLM/data/txt_data"
    input_folder = "/mnt/data/skanda/MSc_IRD_LLM/data/txt_data"
    output_folder = "/mnt/data/skanda/MSc_IRD_LLM/data/data_preprocessed"
    process_files(input_folder, output_folder, batch_size=10)
    print("Preprocessing complete.")


Processing batch 1 of 6


Batch 1:   0%|          | 0/10 [00:00<?, ?it/s]


Processing file: 10.1177_2515841420954592.txt
Starting preprocessing...
Initial word count: 14885
Abstract extracted. Abstract word count: 177
Main text word count: 14313
After reassemble_hyphenated_words: 14281 words
After remove_figures_tables: 14280 words
After remove_citations: 13116 words
After remove_urls: 12923 words
After remove_emails: 12918 words
After remove_numerical_references: 12918 words
After remove_headers: 12475 words
After remove_metadata: 12475 words
After remove_institution_names: 12475 words
After remove_copyright_info: 12475 words
After remove_doi_and_journal_info: 12471 words
After remove_artifacts: 12471 words
After removing extra whitespace: 12471 words
After handle_special_characters: 12447 words


Batch 1:  10%|█         | 1/10 [00:02<00:23,  2.58s/it]

After detect_sentence_boundaries: 12447 words
Final preprocessed word count: 12447
Percentage of words retained after cleaning: 83.62%

Processing file: 13287_2023_Article_3526.txt
Starting preprocessing...
Initial word count: 6492
Abstract extracted. Abstract word count: 199
Main text word count: 6159
After reassemble_hyphenated_words: 6135 words
After remove_figures_tables: 6135 words
After remove_citations: 5663 words
After remove_urls: 5648 words
After remove_emails: 5645 words
After remove_numerical_references: 5645 words
After remove_headers: 5645 words
After remove_metadata: 5641 words
After remove_institution_names: 5641 words
After remove_copyright_info: 5616 words
After remove_doi_and_journal_info: 5616 words
After remove_artifacts: 5616 words
After removing extra whitespace: 5616 words
After handle_special_characters: 5600 words


...

After detect_sentence_boundaries: 5255 words
Final preprocessed word count: 5255
Percentage of words retained after cleaning: 77.63%
Completed processing batch 5

Processing batch 6 of 6


Batch 6:   0%|          | 0/1 [00:00<?, ?it/s]


Processing file: biomolecules-13-00271.txt
Starting preprocessing...
Initial word count: 31269
Abstract extracted. Abstract word count: 118
Main text word count: 30950
After reassemble_hyphenated_words: 30866 words
After remove_figures_tables: 30866 words
After remove_citations: 30230 words
After remove_urls: 30212 words
After remove_emails: 30212 words
After remove_numerical_references: 30212 words
After remove_headers: 30212 words
After remove_metadata: 30212 words
After remove_institution_names: 30212 words
After remove_copyright_info: 30212 words
After remove_doi_and_journal_info: 30212 words
After remove_artifacts: 30212 words
After removing extra whitespace: 30212 words
After handle_special_characters: 30125 words


Batch 6: 100%|██████████| 1/1 [00:06<00:00,  6.57s/it]

After detect_sentence_boundaries: 30125 words
Final preprocessed word count: 30125
Percentage of words retained after cleaning: 96.34%
Completed processing batch 6
All batches processed.
Preprocessing complete.
